In [1]:
import os
import pydub
import matplotlib.pyplot as plt
import shutil
from tempfile import mktemp
from opensoundscape.audio import Audio
from opensoundscape.spectrogram import Spectrogram
from IPython.display import clear_output
import random

playlist download

In [2]:
pl_link = "https://open.spotify.com/playlist/37i9dQZF1DXaXB8fQg7xif?si=9898256e6779497c"

playlist_dl_dir = "D:\\uni\\deepLearning\\dataset_2"
dl_string = f"spotify_dl -o {playlist_dl_dir}"
os.system(dl_string+f" -l {pl_link}")

renaming files

In [ ]:
def rename_files(folder):
    if not os.path.exists(os.path.join(folder, "renamed")):
        os.mkdir(os.path.join(folder, "renamed"))
    files = [f for f in os.listdir(folder)]
    for count, name in enumerate(files):
        if name.endswith(".txt") or os.path.isdir(os.path.join(folder, name)):
            continue
        new_name = f"audio_{count}.mp3"
        shutil.copy(os.path.join(folder, name), os.path.join(folder, "renamed", new_name))

files shortening

In [4]:
def shorten_audio(folder, audio_len=0.5):
    renamed_folder = os.path.join(folder, "renamed")
    renamed_files = os.listdir(renamed_folder)
    shortened_folder = os.path.join(folder, "shortened")
    if not os.path.exists(shortened_folder):
        os.mkdir(shortened_folder)
    for count, file in enumerate(renamed_files):
        filePath = os.path.join(renamed_folder, file)
        saveFilepath = os.path.join(shortened_folder, str(count))
        if os.path.exists(saveFilepath+"_0.mp3") or not os.path.isfile(filePath):
            continue
        sound = pydub.AudioSegment.from_mp3(filePath)
        for n in range(3):
            save_path = f"{saveFilepath}_{str(n)}.mp3"
            if os.path.exists(save_path):
                continue
            extract = sound[0+(0.5*60*1000*n):audio_len*60*1000+(audio_len*60*1000*n)]
            extract.export(save_path, format="mp3")
        clear_output(wait=True)
        print(f"{count}/{len(renamed_files)}")
    print(f"shortening {folder} done!")

Spectrogram generation

In [5]:
def generate_spectrograms(folder, spectrogram_shape=(200, 500)):
    shortened_folder = os.path.join(folder, "shortened")
    spectrogram_folder = os.path.join(folder, "spectrogram")
    if not os.path.exists(spectrogram_folder):
        os.mkdir(spectrogram_folder)
    files = os.listdir(shortened_folder)
    for count, file in enumerate(files):
        audio_path = os.path.join(shortened_folder, file)
        im_save_path = os.path.join(spectrogram_folder, f"{count+1}.png")
        audio = Audio.from_file(audio_path)
        spectrogram = Spectrogram.from_audio(audio)
        image = spectrogram.to_image(spectrogram_shape)
        image.save(im_save_path)
        clear_output(wait=True)
        print(f"{count+1}/{len(files)}")
    print(f"all spectrograms in {folder} created")


Dataset creation

In [ ]:
main_folder = "D:\\uni\\deepLearning\\dataset_2"
ganre_names = [f for f in os.listdir(main_folder) if not os.path.isfile(os.path.join(main_folder, f))]
train_val_split = (8/10)

for g in ganre_names:
    ganre_folder = os.path.join(main_folder, g)
    if g == "classical" or g == "dance":
        continue

    shorten_audio(ganre_folder)
    generate_spectrograms(ganre_folder)

    spectrogram_folder = os.path.join(ganre_folder, "spectrogram")
    spectrogram_files = [f for f in os.listdir(spectrogram_folder) if os.path.isfile(os.path.join(spectrogram_folder, f))]

    train_dir = os.path.join(spectrogram_folder, "train")
    test_dir = os.path.join(spectrogram_folder, "test")
    if not os.path.exists(train_dir) or not os.path.exists(test_dir):
        os.makedirs(train_dir)
        os.makedirs(test_dir)
    
    random.shuffle(spectrogram_files)
    train_files = spectrogram_files[0:int(len(spectrogram_files)*train_val_split)]
    for f in train_files:
        shutil.move(os.path.join(spectrogram_folder, f), os.path.join(train_dir, f))
        spectrogram_files.remove(f)
    for f in spectrogram_files:
        shutil.move(os.path.join(spectrogram_folder, f), os.path.join(test_dir, f))
    print(f"Splitting done in {g} folder")
    